In [0]:
pip install mlflow==2.11.4

In [0]:
import pandas as pd
import numpy as np
import boto3
import io
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import tempfile
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from pyspark.sql.functions import concat_ws, col


In [0]:
s3 = boto3.client('s3')
bucket = "columbia-gr5069-main"

# Define a helper to read S3 CSV
def read_s3_csv(key):
    obj = s3.get_object(Bucket=bucket, Key=key)
    return pd.read_csv(io.BytesIO(obj['Body'].read()))

# Load datasets from S3
results = read_s3_csv("raw/results.csv") #info on rank, fastest lap time and speed
races = read_s3_csv("raw/races.csv") #dates and names of all the races but has a lot of empty data
drivers = read_s3_csv("raw/drivers.csv") #drivers names and nationalities
lap_times = read_s3_csv("raw/lap_times.csv") #for each race, for each driver, the lap time, the number of laps and the time of the fastest lap
pit_stops = read_s3_csv("raw/pit_stops.csv") #time spent at the pitstop in each lap
qualifying = read_s3_csv("raw/qualifying.csv") #time taken in each round-the ones who dont finish or are disqualified are /N

want to see what each dataset means

In [0]:
display (results)
     

display (races)
     

display(drivers)
     

display(lap_times)

     

display(pit_stops)
     

display(qualifying)

Preparing dataset by combining results with pitstops

In [0]:

%python
# Reset the index of the results DataFrame
results_reset = results.reset_index()

# Join results df with pitstops df with suffixes for overlapping columns
pitstop_results_df = pit_stops.join(
    results_reset.set_index(['raceId', 'driverId']),
    on=['raceId', 'driverId'],
    how='inner',
    lsuffix='_pitstop',
    rsuffix='_result'
)

display(pitstop_results_df)

In [0]:
#get a list of all columns in the dataset
df = pitstop_results_df.columns
print(df)

     
positionOrder: Final race classification (e.g., 1 = winner).

rank: Ranking of the driver's fastest lap in the race (e.g., 1 = fastest lap overall).

fastestLap: Lap number where the driver set their fastest lap.

Objective: I want to predict position order using pitstop_results_df
Feature Selection
Using relevant columns from the dataset:
Pre-Race Features:
grid (starting position), constructorId (team), driverId (driver skill).

In-Race Features:
laps (completed laps), statusId (DNF flag), fastestLapSpeed, fastestLapTime.

Pit Stop Features:
stop (number of pit stops), milliseconds_pitstop (total pit time).

In [0]:
# Select features for modeling
model_data = pitstop_results_df[['grid','constructorId','raceId','driverId','laps','statusId','fastestLapTime', 'fastestLapSpeed','stop','milliseconds_pitstop','rank','positionOrder']]
                            
display(model_data)

I'm only interested in those who finsihed the race, so statusId=1 is the only valid metric for me


In [0]:
model_data['is_DNF'] = (model_data['statusId'] != 1).astype(int)  # 1=Finished, 0=DNF
display(model_data)

In [0]:

#want to check the datatypes and if i have any nans
print(model_data.dtypes)
print(model_data.isna().sum())

Random Forest

In [0]:
%python
df=model_data
# Step 1: Replace '\N' with NaN in problematic columns
cols_with_N = [
    'grid','constructorId','raceId','driverId','laps','statusId','fastestLapTime', 'fastestLapSpeed','stop','milliseconds_pitstop','rank','positionOrder'
]
cols_existing = [col for col in cols_with_N if col in model_data.columns]
model_data[cols_existing] = model_data[cols_existing].replace(r'\N', np.nan)

# Step 3: Drop non-numeric columns OR encode them if needed
df = model_data.select_dtypes(include=[np.number])  

# Step 4: Drop remaining rows with missing data 
df = df.dropna()

#Model Prep

# Step 5: Define features and target
X = df.drop(columns=["positionOrder"])
y = df["positionOrder"]

# Step 6: Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)



In [0]:
with mlflow.start_run(run_name='RandomForest') as run:
  # Create model, train it, and create predictions
  rf = RandomForestRegressor(n_estimators=100, max_depth=10)
  rfFit= rf.fit(X_train, y_train)
  predictionsRF = rf.predict(X_test)

  # Log model
  mlflow.sklearn.log_model(rf, "RandomForestRegressor-model")

  # Create metrics
  rmse_rf = mean_squared_error(y_test, predictionsRF, squared=False)
  mse_rf = mean_squared_error(y_test, predictionsRF)
  mae_rf = mean_absolute_error(y_test, predictionsRF)
  r2_rf = r2_score(y_test, predictionsRF)

  
  # Log model and metrics
  mlflow.sklearn.log_model(rfFit, "random_forest_model")
  mlflow.log_metric("rmse", rmse_rf)
  mlflow.log_metric("r2", r2_rf)
  mlflow.log_metric("mae", mae_rf)
  mlflow.log_metric("mse", mse_rf)
  mlflow.log_param("model_type", "RandomForestRegressor")
  mlflow.log_param("numTrees", 100)
  mlflow.log_param("maxDepth", 10)
  # Saving and logging prediction CSV
  predRF_final = pd.DataFrame({
    'features': X_test.values.tolist(),
    'target': y_test,
    'prediction': predictionsRF
  })
  rf_csv_path = "/tmp/rf_predictions.csv"
  predRF_final.to_csv(rf_csv_path, index=False)
  mlflow.log_artifact(rf_csv_path)

In [0]:
#Using a linear regression with ML Flow to predict the target 'positionOrder'.
mlflow.end_run() 
with mlflow.start_run(run_name="LinearRegression"):
    lr = LinearRegression()
    lrFit= lr.fit(X_train, y_train)
    predictionsLR = lr.predict(X_test)
    # Log model
    mlflow.sklearn.log_model(lr, "LinearRegression-model")

    # Create metrics
    rmse_lr = mean_squared_error(y_test, predictionsLR, squared=False)
    mse_lr = mean_squared_error(y_test, predictionsLR)
    mae_lr = mean_absolute_error(y_test, predictionsLR)
    r2_lr = r2_score(y_test, predictionsLR)

    # Log model and metrics
    mlflow.sklearn.log_model(lrFit, "linear_regression_model")
    mlflow.log_metric("rmse", rmse_lr)
    mlflow.log_metric("r2", r2_lr)
    mlflow.log_metric("mae", mae_lr)
    mlflow.log_metric("mse", mse_lr)
    mlflow.log_param("model_type", "LinearRegression")

    #Saving and logging prediction CSV as a second artifact
    predLR_final = pd.DataFrame({
        'features': X_test.values.tolist(),
        'target': y_test,
        'prediction': predictionsLR
    })
    lr_csv_path = "/tmp/lr_predictions.csv"
    predLR_final.to_csv(lr_csv_path, index=False)
    mlflow.log_artifact(lr_csv_path)
     

In [0]:
# Convert Pandas DataFrames to Spark DataFrames
predRF_final_spark = spark.createDataFrame(predRF_final)
predLR_final_spark = spark.createDataFrame(predLR_final)

# Convert array columns to strings (assuming 'features' is the array column)
predRF_final_spark = predRF_final_spark.withColumn(
    'features', concat_ws(',', col('features'))
)
predLR_final_spark = predLR_final_spark.withColumn(
    'features', concat_ws(',', col('features'))
)

# Saving predictions to tables
predRF_final_spark.write.format('jdbc').options(
    url='jdbc:mysql://as7475-gr5069.ccqalx6jsr2n.us-east-1.rds.amazonaws.com/gr5069',
    driver='com.mysql.jdbc.Driver',
    dbtable='rf_model_predictions',
    user='admin',
    password='amnashahid99'
).mode('overwrite').save()


predLR_final_spark.write.format('jdbc').options(
    url='jdbc:mysql://as7475-gr5069.ccqalx6jsr2n.us-east-1.rds.amazonaws.com/gr5069',
    driver='com.mysql.jdbc.Driver',
    dbtable='lr_model_predictions',
    user='admin',
    password='amnashahid99'
).mode('overwrite').save()
     

In [0]:
# Viewing Random Forest predictions
spark.read.format("jdbc").option("url", "jdbc:mysql://as7475-gr5069.ccqalx6jsr2n.us-east-1.rds.amazonaws.com/gr5069") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("dbtable", "rf_model_predictions") \
    .option("user", "admin") \
    .option("password", "amnashahid99") \
    .load().display()

# Viewing Linear Regression predictions
spark.read.format("jdbc").option("url", "jdbc:mysql://as7475-gr5069.ccqalx6jsr2n.us-east-1.rds.amazonaws.com/gr5069") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("dbtable", "lr_model_predictions") \
    .option("user", "admin") \
    .option("password", "amnashahid99") \
    .load().display()